In [34]:
import pandas as pd

# Load dataset
faa_df = pd.read_csv("faa_ai_prelim.csv")

# View first 5 rows
print("First 5 rows:")
display(faa_df.head())

# Dataset info
print("\nDataset Info:")
print(faa_df.info())

# Missing value count
print("\nMissing Values:")
print(faa_df.isnull().sum())


First 5 rows:


UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0      No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
1      No  19-FEB-16      18-FEB-16      23:55:00Z     TAVERNIER   
2      No  19-FEB-16      18-FEB-16      22:14:00Z       TRENTON   
3      No  19-FEB-16      18-FEB-16      17:10:00Z     ASHEVILLE   
4      No  19-FEB-16      18-FEB-16      00:26:00Z     TALKEETNA   

   LOC_STATE_NAME LOC_CNTRY_NAME  \
0  North Carolina            NaN   
1         Florida            NaN   
2      New Jersey            NaN   
3  North Carolina            NaN   
4          Alaska            NaN   

                                            RMK_TEXT EVENT_TYPE_DESC  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...        Accident   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...        Incident   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...        Incident   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...        Incident   
4  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...        Incident   

                  FSDO_DESC  ... PAX_INJ_NONE PAX_INJ_MINOR PAX_INJ_SERIOUS  \
0     FAA Charlotte FSDO-68  ...          NaN           NaN             NaN   
1         FAA Miami FSDO-19  ...          NaN           NaN             NaN   
2  FAA Philadelphia FSDO-17  ...          NaN           NaN             NaN   
3     FAA Charlotte FSDO-68  ...          NaN           NaN             NaN   
4     FAA Anchorage FSDO-03  ...          NaN           1.0             NaN   

  PAX_INJ_FATAL PAX_INJ_UNK  GRND_INJ_NONE GRND_INJ_MINOR GRND_INJ_SERIOUS  \
0           NaN         NaN            NaN            NaN              NaN   
1           NaN         NaN            NaN            NaN              NaN   
2           NaN         NaN            NaN            NaN              NaN   
3           NaN         NaN            NaN            NaN              NaN   
4           NaN         NaN            NaN            NaN              NaN   

  GRND_INJ_FATAL  GRND_INJ_UNK  
0            NaN           NaN  
1            NaN           NaN  
2            NaN           NaN  
3            NaN           NaN  
4            NaN           NaN  

[5 rows x 42 columns]


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 42 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   UPDATED              83 non-null     object 
 1   ENTRY_DATE           83 non-null     object 
 2   EVENT_LCL_DATE       83 non-null     object 
 3   EVENT_LCL_TIME       83 non-null     object 
 4   LOC_CITY_NAME        83 non-null     object 
 5   LOC_STATE_NAME       83 non-null     object 
 6   LOC_CNTRY_NAME       2 non-null      object 
 7   RMK_TEXT             83 non-null     object 
 8   EVENT_TYPE_DESC      83 non-null     object 
 9   FSDO_DESC            81 non-null     object 
 10  REGIST_NBR           83 non-null     object 
 11  FLT_NBR              6 non-null      object 
 12  ACFT_OPRTR           7 non-null      object 
 13  ACFT_MAKE_NAME       78 non-null     object 
 14  ACFT_MODEL_NAME      79 non-null     object 
 15  ACFT_MISSING_FLAG    0 non-

### onverting date and time columns to a single datetime object


In [35]:
faa_df['EVENT_DATETIME'] = pd.to_datetime(
    faa_df['EVENT_LCL_DATE'] + ' ' + faa_df['EVENT_LCL_TIME'],
    errors='coerce',
    format='%d-%b-%y %H:%M:%SZ'
)

display(faa_df[['EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'EVENT_DATETIME']].head())


EVENT_LCL_DATE EVENT_LCL_TIME      EVENT_DATETIME
0      19-FEB-16      00:45:00Z 2016-02-19 00:45:00
1      18-FEB-16      23:55:00Z 2016-02-18 23:55:00
2      18-FEB-16      22:14:00Z 2016-02-18 22:14:00
3      18-FEB-16      17:10:00Z 2016-02-18 17:10:00
4      18-FEB-16      00:26:00Z 2016-02-18 00:26:00

### Extract the attributes and do taskes

In [36]:

required_columns = [
    'ACFT_MAKE_NAME',     
    'LOC_STATE_NAME',     
    'ACFT_MODEL_NAME',    
    'RMK_TEXT',           
    'FLT_PHASE',          
    'EVENT_TYPE_DESC',    
    'FATAL_FLAG'          
]

faa_selected = faa_df[required_columns].copy()

display(faa_selected.head())


ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0          BEECH  North Carolina              36   
1           VANS         Florida             RV7   
2         CESSNA      New Jersey             172   
3        LANCAIR  North Carolina             235   
4         CESSNA          Alaska             172   

                                            RMK_TEXT       FLT_PHASE  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   UNKNOWN (UNK)   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   LANDING (LDG)   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...  APPROACH (APR)   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...   LANDING (LDG)   
4  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...   LANDING (LDG)   

  EVENT_TYPE_DESC FATAL_FLAG  
0        Accident        Yes  
1        Incident        NaN  
2        Incident        NaN  
3        Incident        NaN  
4        Incident        NaN

#### Replacing all Fatal Flag missing values with no

In [37]:
# Replace missing values in FATAL_FLAG with 'No'
faa_selected['FATAL_FLAG'] = faa_selected['FATAL_FLAG'].fillna('No')

# Verify replacement
print("Missing values in Fatal Flag after replacement:", faa_selected['FATAL_FLAG'].isnull().sum())
display(faa_selected.head())


Missing values in Fatal Flag after replacement: 0


ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0          BEECH  North Carolina              36   
1           VANS         Florida             RV7   
2         CESSNA      New Jersey             172   
3        LANCAIR  North Carolina             235   
4         CESSNA          Alaska             172   

                                            RMK_TEXT       FLT_PHASE  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   UNKNOWN (UNK)   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   LANDING (LDG)   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...  APPROACH (APR)   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...   LANDING (LDG)   
4  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...   LANDING (LDG)   

  EVENT_TYPE_DESC FATAL_FLAG  
0        Accident        Yes  
1        Incident         No  
2        Incident         No  
3        Incident         No  
4        Incident         No

#### Fill missing FLT_PHASE and ACFT_DMG_DESC using the mode 

In [38]:

faa_df['FLT_PHASE'].fillna(faa_df['FLT_PHASE'].mode()[0], inplace=True)

faa_df['ACFT_DMG_DESC'].fillna(faa_df['ACFT_DMG_DESC'].mode()[0], inplace=True)

# Verify missing values replaced
print("Missing values after filling with mode:")
print(faa_df[['FLT_PHASE', 'ACFT_DMG_DESC']].isnull().sum())

# Check the number of observations
print("\nNumber of observations before cleaning:", len(faa_df))


Missing values after filling with mode:
FLT_PHASE        0
ACFT_DMG_DESC    0
dtype: int64

Number of observations before cleaning: 83


C:\Users\Admin\AppData\Local\Temp\ipykernel_332\3067632194.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  faa_df['FLT_PHASE'].fillna(faa_df['FLT_PHASE'].mode()[0], inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_332\3067632194.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 

### Remove observations where aircraft names are not available , columns with more than 75 missing values and checking 

In [39]:

faa_df = faa_df.dropna(subset=['ACFT_MAKE_NAME'])

faa_df = faa_df.dropna(axis=1, thresh=len(faa_df) - 75)

print("Number of observations after cleaning:", len(faa_df))


Number of observations after cleaning: 78


### Group the dataset by aircraft name and view counts

In [40]:

aircraft_counts = faa_df.groupby('ACFT_MAKE_NAME').size().reset_index(name='Count')

print("Number of times each aircraft type appears in the dataset:")
display(aircraft_counts)


Number of times each aircraft type appears in the dataset:


ACFT_MAKE_NAME  Count
0           AERO COMMANDER      1
1                  AERONCA      1
2   AEROSTAR INTERNATIONAL      1
3                   AIRBUS      1
4                    BEECH      9
5                     BELL      2
6                   BOEING      3
7                   CESSNA     23
8                 CHAMPION      2
9                 CHRISTEN      1
10     CONSOLIDATED VULTEE      1
11                 EMBRAER      1
12                 ENSTROM      1
13               FAIRCHILD      1
14           FLIGHT DESIGN      1
15                   GLOBE      1
16             GREAT LAKES      1
17                 GRUMMAN      1
18              GULFSTREAM      1
19                  HUGHES      1
20                 LANCAIR      2
21                   MAULE      1
22                  MOONEY      4
23          NORTH AMERICAN      1
24                   PIPER     10
25                   PITTS      1
26                    SAAB      1
27              SABRELINER      1
28                  SOCATA      2
29                    VANS      1

### Display observations where Fatal Flag is "Yes"

In [41]:
fatal_yes = faa_df[faa_df['FATAL_FLAG'].str.strip().str.upper() == 'YES']
print("Observations with Fatal Flag = 'Yes':")
display(fatal_yes)


Observations with Fatal Flag = 'Yes':


UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0       No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
53      No  12-FEB-16      11-FEB-16      11:32:00Z        DESTIN   
55      No  11-FEB-16      11-FEB-16      00:17:00Z        TULARE   
79      No  08-FEB-16      05-FEB-16      11:17:00Z      MARICOPA   
80      No  08-FEB-16      05-FEB-16      23:02:00Z     SAN PEDRO   
81      No  08-FEB-16      05-FEB-16      23:02:00Z     SAN PEDRO   
82     Yes  02-FEB-16      02-FEB-16      01:52:00Z        MOBILE   

    LOC_STATE_NAME                                           RMK_TEXT  \
0   North Carolina  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   
53         Florida  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES. ...   
55      California  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES A...   
79         Arizona  AIRCRAFT CRASHED UNDER UNKNOWN CIRCUMSTANCES, ...   
80      California  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...   
81      California  N9872R, BEECH M35 AIRCRAFT, AND N5057G, BELLAN...   
82         Alabama  N784CP AIRCRAFT CRASHED INTO A WOODED AREA NEA...   

   EVENT_TYPE_DESC               FSDO_DESC REGIST_NBR  ... FLT_ACTIVITY  \
0         Accident   FAA Charlotte FSDO-68      N61WB  ...          NaN   
53        Accident                     NaN     N2209W  ...          NaN   
55        Accident      FAA Fresno FSDO-17     N911TS  ...        Other   
79        Accident  FAA Scottsdale FSDO-07     N551JP  ...          NaN   
80        Accident  FAA Long Beach FSDO-05     N5057G  ...          NaN   
81        Accident  FAA Long Beach FSDO-05     N9872R  ...          NaN   
82        Accident  FAA Birmingham FSDO-09     N784CP  ...          NaN   

        FLT_PHASE MAX_INJ_LVL FATAL_FLAG FLT_CRW_INJ_NONE FLT_CRW_INJ_MINOR  \
0   UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
53  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
55  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
79  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
80  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
81  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   
82  UNKNOWN (UNK)       Fatal        Yes              NaN               NaN   

   FLT_CRW_INJ_FATAL PAX_INJ_NONE PAX_INJ_SERIOUS      EVENT_DATETIME  
0                1.0          NaN             NaN 2016-02-19 00:45:00  
53               1.0          NaN             NaN 2016-02-11 11:32:00  
55               2.0          NaN             NaN 2016-02-11 00:17:00  
79               2.0          NaN             NaN 2016-02-05 11:17:00  
80               1.0          NaN             NaN 2016-02-05 23:02:00  
81               2.0          NaN             NaN 2016-02-05 23:02:00  
82               2.0          NaN             NaN 2016-02-02 01:52:00  

[7 rows x 25 columns]

### Performing One-hot encode ACFT_DMG_DESC column and merging encoded columns in to dataframe

In [42]:
acft_dmg_encoded = pd.get_dummies(faa_df['ACFT_DMG_DESC'], prefix='DMG', drop_first=True)

faa_df = pd.concat([faa_df, acft_dmg_encoded], axis=1)

print("Columns after one-hot encoding:")
print(faa_df.columns.tolist())
faa_df.head(5)


Columns after one-hot encoding:
['UPDATED', 'ENTRY_DATE', 'EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'LOC_CITY_NAME', 'LOC_STATE_NAME', 'RMK_TEXT', 'EVENT_TYPE_DESC', 'FSDO_DESC', 'REGIST_NBR', 'FLT_NBR', 'ACFT_OPRTR', 'ACFT_MAKE_NAME', 'ACFT_MODEL_NAME', 'ACFT_DMG_DESC', 'FLT_ACTIVITY', 'FLT_PHASE', 'MAX_INJ_LVL', 'FATAL_FLAG', 'FLT_CRW_INJ_NONE', 'FLT_CRW_INJ_MINOR', 'FLT_CRW_INJ_FATAL', 'PAX_INJ_NONE', 'PAX_INJ_SERIOUS', 'EVENT_DATETIME', 'DMG_Minor', 'DMG_Substantial', 'DMG_Unknown']


UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0      No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
1      No  19-FEB-16      18-FEB-16      23:55:00Z     TAVERNIER   
2      No  19-FEB-16      18-FEB-16      22:14:00Z       TRENTON   
3      No  19-FEB-16      18-FEB-16      17:10:00Z     ASHEVILLE   
4      No  19-FEB-16      18-FEB-16      00:26:00Z     TALKEETNA   

   LOC_STATE_NAME                                           RMK_TEXT  \
0  North Carolina  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   
1         Florida  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   
2      New Jersey  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...   
3  North Carolina  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...   
4          Alaska  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...   

  EVENT_TYPE_DESC                 FSDO_DESC REGIST_NBR  ... FATAL_FLAG  \
0        Accident     FAA Charlotte FSDO-68      N61WB  ...        Yes   
1        Incident         FAA Miami FSDO-19     N705RP  ...        NaN   
2        Incident  FAA Philadelphia FSDO-17     N9563A  ...        NaN   
3        Incident     FAA Charlotte FSDO-68      N10CF  ...        NaN   
4        Incident     FAA Anchorage FSDO-03     N1530Y  ...        NaN   

  FLT_CRW_INJ_NONE FLT_CRW_INJ_MINOR FLT_CRW_INJ_FATAL PAX_INJ_NONE  \
0              NaN               NaN               1.0          NaN   
1              1.0               NaN               NaN          NaN   
2              1.0               NaN               NaN          NaN   
3              1.0               NaN               NaN          NaN   
4              1.0               NaN               NaN          NaN   

  PAX_INJ_SERIOUS      EVENT_DATETIME DMG_Minor DMG_Substantial  DMG_Unknown  
0             NaN 2016-02-19 00:45:00     False           False         True  
1             NaN 2016-02-18 23:55:00     False           False         True  
2             NaN 2016-02-18 22:14:00      True           False        False  
3             NaN 2016-02-18 17:10:00     False           False         True  
4             NaN 2016-02-18 00:26:00      True           False        False  

[5 rows x 28 columns]

In [43]:
# faa_df['DMG_Substantial']=faa_df['DMG_Substantial'].map({True:1,False:0})
# faa_df['DMG_Unknown']=faa_df['DMG_Unknown'].map({True:1,False:0})

faa_df['DMG_Substantial'] = faa_df['DMG_Substantial'].astype(int)
faa_df['DMG_Unknown'] = faa_df['DMG_Unknown'].astype(int)
faa_df['DMG_Minor'] = faa_df['DMG_Minor'].astype(int)

In [44]:
faa_df.head(2)

UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0      No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
1      No  19-FEB-16      18-FEB-16      23:55:00Z     TAVERNIER   

   LOC_STATE_NAME                                           RMK_TEXT  \
0  North Carolina  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   
1         Florida  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   

  EVENT_TYPE_DESC              FSDO_DESC REGIST_NBR  ... FATAL_FLAG  \
0        Accident  FAA Charlotte FSDO-68      N61WB  ...        Yes   
1        Incident      FAA Miami FSDO-19     N705RP  ...        NaN   

  FLT_CRW_INJ_NONE FLT_CRW_INJ_MINOR FLT_CRW_INJ_FATAL PAX_INJ_NONE  \
0              NaN               NaN               1.0          NaN   
1              1.0               NaN               NaN          NaN   

  PAX_INJ_SERIOUS      EVENT_DATETIME DMG_Minor DMG_Substantial  DMG_Unknown  
0             NaN 2016-02-19 00:45:00         0               0            1  
1             NaN 2016-02-18 23:55:00         0               0            1  

[2 rows x 28 columns]

# Advance tasks

In [45]:
# Define keywords for flight phases
phase_keywords = {
    'LANDING': 'LANDING',
    'TAKEOFF': 'TAKEOFF',
    'CRUISE': 'CRUISE',
    'APPROACH': 'APPROACH'
}

# Function to extract flight phase from RMK_TEXT
def extract_flight_phase(text):
    if pd.isnull(text):
        return 'UNKNOWN'
    text_upper = text.upper()
    for keyword, phase in phase_keywords.items():
        if keyword in text_upper:
            return phase
    return 'UNKNOWN'

# Apply extraction to create FLIGHT_PHASE_TEXT
faa_df['FLIGHT_PHASE_TEXT'] = faa_df['RMK_TEXT'].apply(extract_flight_phase)

# Compare with existing FLT_PHASE
print("Comparison of FLT_PHASE vs FLIGHT_PHASE_TEXT:")
display(faa_df[['FLT_PHASE', 'FLIGHT_PHASE_TEXT']].head())


Comparison of FLT_PHASE vs FLIGHT_PHASE_TEXT:


FLT_PHASE FLIGHT_PHASE_TEXT
0   UNKNOWN (UNK)           UNKNOWN
1   LANDING (LDG)           LANDING
2  APPROACH (APR)           UNKNOWN
3   LANDING (LDG)           LANDING
4   LANDING (LDG)           LANDING